In [11]:
# imports
import wget
import zipfile
import os
import pandas as pd
import shutil

In [25]:
# globals
dataset_url = 'https://api.gbif.org/v1/occurrence/download/request/0230970-210914110416597.zip'
output_dir = '/projectnb/sparkgrp/ml-herbarium-grp/ml-herbarium-angeline1/ml-herbarium/corpus/corpus_geography/'
zip_file = 'asia_and_oceania.zip'
txt = 'occurrence.txt'
txt_dir = 'asia_and_oceania/occurrence.txt'
csv = 'occurrence.csv'
corpus_geography = 'corpus_geography.txt'

# Download Dataset

In [ ]:
# if os.path.exists(output_dir + zip_file):
#     os.remove(output_dir + zip_file)
# wget.download(dataset_url, out=output_dir + zip_file)

# Extract files from .zip

In [52]:
if os.path.exists(output_dir + 'asia_and_oceania'):
    shutil.rmtree(output_dir + 'asia_and_oceania')
with zipfile.ZipFile(output_dir + zip_file,'r') as zip_ref:
    zip_ref.extractall(output_dir + 'asia_and_oceania') 

# Convert .txt to .csv

In [55]:
read_file = pd.read_csv(output_dir + txt_dir, sep='\t')
read_file.to_csv(output_dir + csv)
df = pd.read_csv(csv, low_memory=False)
print(df['higherGeography'])

/Users/angelinetseng/opt/anaconda3/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3444: DtypeWarning: Columns (75,91,136,172,175,187,195,201,205,208,212,215,224,228,238,242) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


0                           Asia;China;Xizang (Tibet);
1                          Asia;China;Yunnan;Pingbian;
2                                          Asia;Japan;
3         Oceania;Australia (Country);New South Wales;
4                            Asia;Philippines;Palawan;
                              ...                     
163504                            Oceania;New Zealand;
163505                              Asia;China;Yunnan;
163506                       Asia;Philippines;Palawan;
163507                       Asia;China;Yunnan;Lushui;
163508               Oceania;Papua New Guinea;Western;
Name: higherGeography, Length: 163509, dtype: object


In [56]:
higherGeography = df['higherGeography'] #you can also use df['column_name']
higherGeography = higherGeography.unique().tolist()
print(higherGeography[:5])
print(len(higherGeography))

['Asia;China;Xizang (Tibet);', 'Asia;China;Yunnan;Pingbian;', 'Asia;Japan;', 'Oceania;Australia (Country);New South Wales;', 'Asia;Philippines;Palawan;']
1467


In [57]:
geo_no_semi = []
for i in range(len(higherGeography)): 
    geo_no_semi.append(higherGeography[i].split(';')[:-1])
print(geo_no_semi[:5])

[['Asia', 'China', 'Xizang (Tibet)'], ['Asia', 'China', 'Yunnan', 'Pingbian'], ['Asia', 'Japan'], ['Oceania', 'Australia (Country)', 'New South Wales'], ['Asia', 'Philippines', 'Palawan']]


In [58]:
string = ', '.join([str(item) for item in geo_no_semi[18][1:]])
print(string)

Philippines, Benguet


In [59]:
textfile = open(output_dir + corpus_geography, 'w')
for i in range(len(geo_no_semi)):
    textfile.write(', '.join([str(item) for item in geo_no_semi[i][1:]]) + '\n')
textfile.close()